<a href="https://colab.research.google.com/github/dnrb/word2vec_char_level/blob/master/first_file(English_Model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/dnrb/word2vec_char_level.git

Cloning into 'word2vec_char_level'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 59 (delta 27), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (59/59), done.


In [3]:
cd word2vec_char_level/

/content/word2vec_char_level


**Verify the first data file**

In [4]:
file1 = 'tasa_1.txt'
filehandle1 = open(file1)
lines1 = filehandle1.readlines()
len(lines1)

250000

**Verify the second data file**

In [5]:
file2 = 'tasa_2.txt'
filehandle2 = open(file2)
lines2 = filehandle2.readlines()
len(lines2)

250000

**Verify the third data file**

In [6]:
file3 = 'tasa_3.txt'
filehandle3 = open(file1)
lines3 = filehandle3.readlines()
len(lines3)

250000

**Construct the corpus**

In [7]:
corpus = lines1 + lines2 + lines3
len(corpus)

750000



**Gather every words in the corpus**

In [0]:
import string
words = []

for sentence in corpus:
    
    for word in sentence.split(' '):
        
        if word not in string.punctuation:
            words.append(word)
            


In [9]:
len(words)

11556159

In [10]:
words[:10]

['who',
 'were',
 'the',
 'first',
 'americans',
 '?\n',
 'many',
 'many',
 'years',
 'ago']

In [0]:
word_set = set(words)

**Convert text to int**

In [0]:
def str_to_int(lst):
    counter = 0
    new_list = []
    existed = {}
    
    for string in lst:
        if string not in existed:
            existed[string] = counter
            new_list.append(existed[string])
            counter += 1
        else:
            new_list.append(existed[string])
            
    return new_list

In [0]:
word2int = str_to_int(words)

In [14]:
min(word2int)

0

In [15]:
max(word2int)

95209

**Set up the constants**

In [0]:
n_Train = 50
n_Voc = max(word2int) + 1
n_Corpus = len(word2int)
WINDOW_SIZE = 2

In [17]:
n_Voc

95210

**Initialize two arrays**

In [0]:
import numpy as np

def data_generate(n_Train, n_Voc, n_Corpus, WINDOW_SIZE, word2int):

    X = np.zeros((n_Train, n_Voc))
    Y = np.zeros((n_Train, n_Voc))

    for i in range(n_Train):
        j = np.random.choice(n_Corpus, 1)[0]

        tw = word2int[j]

        Y[i][tw] = 1

        for k in range(j - WINDOW_SIZE, j + WINDOW_SIZE + 1):
            if k != j and k >= 0 and k < n_Corpus:
                cw = word2int[k]
                X[i][cw] = 1
                
    return X, Y

In [19]:
data_generate(n_Train, n_Voc, n_Corpus, WINDOW_SIZE, word2int)

(array([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

In [20]:
import tensorflow as tf

ONE_HOT_DIM = n_Voc

# # function to convert numbers to one hot vectors
# def to_one_hot_encoding(data_point_index):
#     one_hot_encoding = np.zeros(ONE_HOT_DIM)
#     one_hot_encoding[data_point_index] = 1
#     return one_hot_encoding

# X = [] # input word
# Y = [] # target word

# for x, y in zip(df['input'], df['label']):
#     X.append(to_one_hot_encoding(word2int[ x ]))
#     Y.append(to_one_hot_encoding(word2int[ y ]))

# # convert them to numpy arrays
# X_train = np.asarray(X)
# Y_train = np.asarray(Y)

# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float64, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float64, shape=(None, ONE_HOT_DIM))

# word embedding will be 2 dimension for 2d visualization
EMBEDDING_DIM = 30

# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM], dtype=tf.float64))
b1 = tf.Variable(tf.random_normal([1], dtype=tf.float64)) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM], dtype=tf.float64))
b2 = tf.Variable(tf.random_normal([1], dtype=tf.float64))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation(learning rate)
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


**Train**

In [22]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 20000
for i in range(iteration + 1):
    X, Y = data_generate(n_Train, n_Voc, n_Corpus, WINDOW_SIZE, word2int)
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X, y_label: Y})
    if i % 500 == 0:
        print('iteration '+str(i)+' loss is : ', sess.run(loss, feed_dict={x: X, y_label: Y}))

iteration 0 loss is :  47.185211897620704
iteration 500 loss is :  37.194145890825425
iteration 1000 loss is :  34.08522663068314
iteration 1500 loss is :  30.99789962617716
iteration 2000 loss is :  31.618508257170603
iteration 2500 loss is :  26.706190519584755
iteration 3000 loss is :  24.5662889929537
iteration 3500 loss is :  25.729650847766823
iteration 4000 loss is :  26.989030869512522
iteration 4500 loss is :  25.086860629470333
iteration 5000 loss is :  25.716319344143226
iteration 5500 loss is :  24.089234606563696
iteration 6000 loss is :  23.981382152955735
iteration 6500 loss is :  22.920383605277088
iteration 7000 loss is :  23.444663737750282
iteration 7500 loss is :  18.196834042602333
iteration 8000 loss is :  20.653580463399816


KeyboardInterrupt: ignored

**Visualization**

In [0]:
import sklearn
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import TSNE

Z = np.array([[0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1]])
Z_embedded = TSNE(n_components=2).fit_transform(Z)
Z_embedded.shape

In [0]:
vectors = sess.run(W1 + b1)
vectors

In [0]:
vector_embedded_2d = TSNE(perplexity=100).fit_transform(vectors)
vector_embedded_2d.shape

In [0]:
len(words)

In [0]:
def tsne_plot_2d(label, embeddings, words=[], a=1):
    plt.figure(figsize=(16, 9))
    colors = cm.rainbow(np.linspace(0, 1, 1))
    x = embeddings[:,0]
    y = embeddings[:,1]
    plt.scatter(x, y, c=colors, alpha=a, label=label)
    for i, word in enumerate(words):
        plt.annotate(word, alpha=0.3, xy=(x[i], y[i]), xytext=(5, 2), 
                     textcoords='offset points', ha='right', va='bottom', size=10)
    plt.legend(loc=4)
    plt.grid(True)
    plt.savefig("hhh.png", format='png', dpi=150, bbox_inches='tight')
    plt.show()

tsne_plot_2d('Visualizing Embeddings using t-SNE', vector_embedded_2d, word_set, a=0.1)

In [0]:
vector_embedded_3d = TSNE(n_components=3).fit_transform(vectors)
vector_embedded_3d.shape

In [0]:
from mpl_toolkits.mplot3d import Axes3D

In [0]:
def tsne_plot_3d(title, label, embeddings, a=1):
    fig = plt.figure()
    ax = Axes3D(fig)
    colors = cm.rainbow(np.linspace(0, 1, 1))
    plt.scatter(embeddings[:, 0], embeddings[:, 1], embeddings[:, 2], c=colors, alpha=a, label=label)
    plt.legend(loc=4)
    plt.title(title)
    plt.show()


tsne_plot_3d('Visualizing Embeddings using t-SNE', 'tasa_1.txt', vector_embedded_3d, a=0.1)

**Remove stop words**

In [0]:
# import nltk
# nltk.download('stopwords')

In [0]:
# stopwords = nltk.corpus.stopwords.words('english')
# cleaned_words = []

# for word in words:
#     if word not in stopwords:
#         cleaned_words.append(word)